Total: 47/50

<b>Jim Fitzgerald 819812741 - Assignment 4</b>
</br>

<hr/>
<b>1 Did age have any affect on the survival of the passengers? Divide the passengers into age
groups spanning 5 years each - [0, 5), [5, 10), [10, 15), ... . For each group compute thenumber of passengers in each group. Then compute the percent of survivors in each
group.</b>

In [1]:
import org.apache.spark.mllib.rdd.RDDFunctions._
import org.apache.spark.sql.functions._
import org.apache.spark.ml.feature.Bucketizer
import org.apache.spark.sql.DataFrameNaFunctions
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.feature.RFormula
import org.apache.spark.mllib.tree.DecisionTree
import org.apache.spark.mllib.tree.model.DecisionTreeModel
import org.apache.spark.sql.types.{StructField, StructType, StringType, DoubleType, IntegerType}
import org.apache.spark.ml.feature.Binarizer


In [2]:
val schema = new StructType(Array(
  new StructField("survived", StringType, true),
  new StructField("sex", StringType, true),
  new StructField("age", IntegerType, true),
  new StructField("pclass", StringType, true),
  new StructField("name", StringType, true),
  new StructField("sibsp", IntegerType, true),
  new StructField("parch", IntegerType, true),    
  new StructField("ticket", StringType, true),    
  new StructField("fare", DoubleType, true),    
  new StructField("cabin", StringType, true),    
  new StructField("embarked", StringType, true),    
  new StructField("boat", StringType, true),    
  new StructField("body", StringType, true),    
  new StructField("destination", StringType, true)
))

val file = "titanic.tsv"
val reader = spark.read
reader.option("header", true)
reader.option("inferSchema", true)
reader.option("delimiter", "\t")
val titanic = reader.csv(file)

In [3]:
// create buckets for age
val splits = (0 to 100).map(_ * 5.0).toArray
val bucketizer = new Bucketizer().
    setInputCol("age").
    setOutputCol("bucket").
    setSplits(splits)

// get the totals for each bucket
val ages = bucketizer.transform(titanic)
var total = ages.selectExpr("bucket", "age", "survived").
    groupBy("bucket").
    agg(
        count("age").as("total")).
    sort("bucket")

// get the survivors for each bucket
val titanicSurvivor = titanic.filter("survived == 'y'")
val survivors = bucketizer.transform(titanicSurvivor)

val survivorsOnly = survivors.groupBy("bucket").
    agg(
        count("age").as("survivors"))

val s2 = survivors.where(col("survived") === "y")

var combined = total.join(survivorsOnly, "bucket").
    withColumn("percent", survivorsOnly("survivors") / total("total") *100)

// combine to 
val display = combined.selectExpr("bucket*5", "total", "survivors", "percent").
    withColumnRenamed("(bucket * 5)", "Age")
display.show

+----+-----+---------+------------------+
| Age|total|survivors|           percent|
+----+-----+---------+------------------+
| 0.0|   50|       32|              64.0|
| 5.0|   31|       17| 54.83870967741935|
|10.0|   27|       11| 40.74074074074074|
|15.0|  116|       45|38.793103448275865|
|20.0|  184|       71| 38.58695652173913|
|25.0|  160|       56|              35.0|
|30.0|  132|       54|40.909090909090914|
|35.0|  100|       44|              44.0|
|40.0|   69|       20|28.985507246376812|
|45.0|   66|       32|48.484848484848484|
|50.0|   43|       21|48.837209302325576|
|55.0|   27|       11| 40.74074074074074|
|60.0|   27|       10| 37.03703703703704|
|75.0|    1|        1|             100.0|
|80.0|    1|        1|             100.0|
+----+-----+---------+------------------+



In [ ]:
-1: Determining correlation would have produced accurate results.

<b><em>It seems that children (under 15) have a good chance to survive.  There is a small spike in the 35-40 age range, which most likely would be accounting for the parents of those children.  Elderly had a 100% but with only 1 data point it does not seem like a reliable metric.</em>
<hr/>2 Logistic on age. Using logistic regression with independent variable age and dependent
variable survived create a model to classify passengers as survivors.</b>

In [4]:
// binary - survived or did not

//establish training and test data
val data = titanic.filter("age >= 0").
    select("age", "survived")
    //withColumnRenamed("survived", "features").show()

val formula = new RFormula().setFormula("survived ~ age")
val fitted = formula.fit(data)
val prepared = fitted.transform(data)

val Array(train, test) = prepared.randomSplit(Array(0.7,0.3))

// to see that training data contains surive and fail
train.groupBy("survived").count().show()


val lr = new LogisticRegression().
    setThreshold(0.5f)
val lrModel2 = lr.fit(train)

println(s"Coefficients: ${lrModel2.coefficients} intercept: ${lrModel2.intercept}")

val results2 = lrModel2.evaluate(test).predictions
//results2.show()

val binarizer: Binarizer = new Binarizer().
  setInputCol("prediction").
  setOutputCol("binarized_prediction").
  setThreshold(0.5)

val predictionBinary2 = binarizer.transform(results2) 

predictionBinary2.show

+--------+-----+
|survived|count|
+--------+-----+
|       n|  425|
|       y|  317|
+--------+-----+

Coefficients: [-0.00893438835712649] intercept: -0.025620550568915948
+------+--------+--------+-----+--------------------+--------------------+----------+--------------------+
|   age|survived|features|label|       rawPrediction|         probability|prediction|binarized_prediction|
+------+--------+--------+-----+--------------------+--------------------+----------+--------------------+
|0.1667|       y|[0.1667]|  1.0|[0.02710991310804...|[0.50677706321668...|       0.0|                 0.0|
|0.6667|       y|[0.6667]|  1.0|[0.03157710728661...|[0.50789362092941...|       0.0|                 0.0|
|  0.75|       n|  [0.75]|  0.0|[0.03232134183676...|[0.50807963209291...|       0.0|                 0.0|
|0.8333|       y|[0.8333]|  1.0|[0.03306557638690...|[0.50826564101939...|       0.0|                 0.0|
|   1.0|       n|   [1.0]|  0.0|[0.03455493892604...|[0.50863787524849...|    

In [ ]:
-1: No need to write comments. Its clear from code.

<hr/>
<b>3 Logistic on age, sex and pclass. Same as problem two but use independent variables sex,
age, and pclass. Since sex and pclass are categorical they need special treatment.</b>

In [5]:
val data = titanic.filter("age >= 0").select("survived", "age", "sex", "pclass")

val formula = new RFormula().setFormula("survived ~ age + sex + pclass")
val fitted = formula.fit(data)
val prepared = fitted.transform(data)

val Array(train, test) = prepared.randomSplit(Array(0.7,0.3))

// to see that training data contains surive and fail
train.groupBy("survived").count().show()


val lr = new LogisticRegression()
val lrModel3 = lr.fit(train)

//println(s"Coefficients: ${lrModel3.coefficients} intercept: ${lrModel3.intercept}")

val results3 = lrModel3.evaluate(test).predictions
//results3.show()

val binarizer: Binarizer = new Binarizer().
  setInputCol("prediction").
  setOutputCol("binarized_prediction").
  setThreshold(0.5)

val predictionBinary3 = binarizer.transform(results3) 

predictionBinary3.select("survived", "age", "sex", "pclass", "binarized_prediction").show

+--------+-----+
|survived|count|
+--------+-----+
|       n|  436|
|       y|  301|
+--------+-----+

+--------+------+------+------+--------------------+
|survived|   age|   sex|pclass|binarized_prediction|
+--------+------+------+------+--------------------+
|       n|0.3333|  male| third|                 0.0|
|       n|   2.0|female| third|                 1.0|
|       n|   2.0|female| third|                 1.0|
|       n|   2.0|  male| third|                 0.0|
|       n|   2.0|  male| third|                 0.0|
|       n|   4.0|  male| third|                 0.0|
|       n|   6.0|  male| third|                 0.0|
|       n|   9.0|female| third|                 1.0|
|       n|  10.0|female| third|                 1.0|
|       n|  11.0|  male| third|                 0.0|
|       n|  11.5|  male| third|                 0.0|
|       n|  13.0|  male| third|                 0.0|
|       n|  14.5|  male| third|                 0.0|
|       n|  16.0|  male| third|                 0

<hr/>
<b>4 Decision tree. Instead of using logistic regression use Decision tree with the independent
variables sex, age, and pclass.</b>

In [6]:
import org.apache.spark.ml.regression.DecisionTreeRegressionModel
import org.apache.spark.ml.regression.DecisionTreeRegressor
import org.apache.spark.ml.feature.Binarizer

val data = titanic.filter("age >= 0").select("survived", "age", "sex", "pclass")

val formula = new RFormula().setFormula("survived ~ age + sex + pclass")
val fitted = formula.fit(data)
val prepared = fitted.transform(data)

val Array(train, test) = prepared.randomSplit(Array(0.7,0.3))

val dt = new DecisionTreeRegressor().setLabelCol("label").setFeaturesCol("features")

val model = dt.fit(train)

val predictions4 = model.transform(test)
//predictions4 gives decimal not binary - binarize


val binarizer: Binarizer = new Binarizer().
  setInputCol("prediction").
  setOutputCol("binarized_prediction").
  setThreshold(0.5)

val predictionBinary4 = binarizer.transform(predictions4) 

predictionBinary4.show()

println("total : " + predictionBinary4.count)
println("label1 : " + predictionBinary4.filter("label==1.0").count)
println("predict1 : " + predictionBinary4.filter("binarized_prediction==1.0".count)
println("total : " + predictionBinary4.count)

+--------+----+------+------+------------------+-----+------------------+--------------------+
|survived| age|   sex|pclass|          features|label|        prediction|binarized_prediction|
+--------+----+------+------+------------------+-----+------------------+--------------------+
|       n| 2.0|  male| third| [2.0,1.0,1.0,0.0]|  0.0|0.3333333333333333|                 0.0|
|       n| 2.0|  male| third| [2.0,1.0,1.0,0.0]|  0.0|0.3333333333333333|                 0.0|
|       n| 2.0|  male| third| [2.0,1.0,1.0,0.0]|  0.0|0.3333333333333333|                 0.0|
|       n| 3.0|female| third| [3.0,0.0,1.0,0.0]|  0.0|0.4782608695652174|                 0.0|
|       n| 4.0|  male| third| [4.0,1.0,1.0,0.0]|  0.0|              0.25|                 0.0|
|       n| 7.0|  male| third| [7.0,1.0,1.0,0.0]|  0.0|              0.25|                 0.0|
|       n| 9.0|female| third| [9.0,0.0,1.0,0.0]|  0.0|0.4782608695652174|                 0.0|
|       n| 9.0|female| third| [9.0,0.0,1.0,0.0]|  

Name: Syntax Error.
Message: 
StackTrace: 

<hr/>
<b>5. How do the models created in problems 2-4 compare based on the false positives & false
negatives the produce on your test data.</b>

In [7]:
def matrix(tp: Double, tn: Double, fp: Double, fn: Double)={
    print("\t\tPredicted\n\t\tY\tN\n")
    print("Actual\tY")
    print("\t" + tp + "\t" + fn + "\n\tN\t" + fp + "\t" + tn + "\n\n")
}

In [8]:
import org.apache.spark.sql.DataFrame

def printMatrix(tp: Double, tn: Double, fp: Double, fn: Double)={
    print("\t\tPredicted\n\t\tY\tN\n")
    print("Actual\tY")
    print("\t" + tp + "\t" + fn + "\n\tN\t" + fp + "\t" + tn + "\n\n")
}

// Accuracy How often is the model correct
def accuracy(tp: Double, tn: Double, total: Double):Double={
    val accuracy = (tp + tn)/total*100
    println("Accuracy: " + accuracy)
    return accuracy
}

// Misclassification Rate How often is the model wrong
def missclassification(fp: Double, fn: Double, total:Double):Double = {
    val missclassify = (fp + fn)/total*100
    println("Missclassification: " + missclassify)
    return missclassify
}

// True Positive Rate or Sensitivity or Recall When actually admit (yes) how often does model predict admit (yes)
def sensitivity(tp:Double, ap:Double):Double = {
    val sense = tp/ap*100
    println("Sensitivity: " + sense)
    return sense
}

// False Positive Rate When it's actually no (deny), how often does it predict yes (admit)
def fpRate(fp:Double, an:Double):Double = {
    val fpR = fp/an*100
    println("FP Rate: " + fpR)
    return fpR
}

// Specificity When it's actually no, how often does it predict no
def specificity(tn:Double, an:Double):Double = {
    val spec = tn/an*100
    println("Specificity: " + spec)
    return spec
}

// Precision When it predicts yes, how often is it correct
def precision(tp:Double, pp:Double):Double = {
    val prec = tp/pp*100
    println("Precision: " + prec)
    return prec
}

// Prevalence How often does the yes condition actually occur in our sample
def prevalence(ap:Double, total:Double):Double = {
    val prev = ap/total*100
    println("Prevalence: " + prev)
    return prev
}

def wrong(df: DataFrame) ={
    val wrongPredictions = df.where(expr("label != binarized_prediction"))
    val countErrors = wrongPredictions.groupBy("label").agg(count("prediction").alias("Errors"))
    countErrors.show
}

def right(df: DataFrame)={
    val correctPredictions = df.where(expr("label == binarized_prediction"))
    val countCorrectPredictions = correctPredictions.groupBy("label").agg(count("prediction").alias("Correct"))
    countCorrectPredictions.show
}

In [10]:
import org.apache.spark.sql.DataFrame

// gives results for a dataframe
// boolean controls long form (all data analysis)
// false for shorter - just matrix, accuracy and missclassifcation
def analysis(df :DataFrame, long:Boolean) ={
    // extract tp, tn, fp, fn
    val tp = df.filter("label == 1.0").filter("binarized_prediction == 1.0").count
    val tn = df.filter("label == 0.0").filter("binarized_prediction == 0.0").count
    // false positive -- predict survive but didnt
    val fp = df.filter("label == 0.0").filter("binarized_prediction == 1.0").count
    //false negative -- survive, but predict not
    val fn = df.filter("label == 1.0").filter("binarized_prediction == 0.0").count
    val total = tp + fp + tn + fn
    // ap, an, pp
    val ap = df.filter("label == 1.0").count
    val an = df.filter("label == 0.0").count
    val pp = df.filter("prediction == 1.0").count
    
    printMatrix(tp, tn, fp, fn)
    accuracy(tp, tn, total)
    missclassification(fp, fn, total)
    wrong(df)
    right(df)
    if (long){
        sensitivity(tp, ap)
        fpRate(fp,an)
        specificity(tn, an)
        precision(tp, pp)
        prevalence(ap, total)
    }    
}

In [11]:
println("Results from #2 - logistic regression of age only")
analysis(predictionBinary2, true)

Results from #2 - logistic regression of age only
		Predicted
		Y	N
Actual	Y	0.0	109.0
	N	0.0	194.0

Accuracy: 64.02640264026402
Missclassification: 35.973597359735976
+-----+------+
|label|Errors|
+-----+------+
|  1.0|   109|
+-----+------+

+-----+-------+
|label|Correct|
+-----+-------+
|  0.0|    194|
+-----+-------+

Sensitivity: 0.0
FP Rate: 0.0
Specificity: 100.0
Precision: NaN
Prevalence: 35.973597359735976


35.973597359735976

In [ ]:
-1: False positives is expected to be non-zero.

In [12]:
println("results from #3 - logisitic regression of age, sex and pclass")
analysis(predictionBinary3, true)

results from #3 - logisitic regression of age, sex and pclass
		Predicted
		Y	N
Actual	Y	88.0	37.0
	N	29.0	154.0

Accuracy: 78.57142857142857
Missclassification: 21.428571428571427
+-----+------+
|label|Errors|
+-----+------+
|  0.0|    29|
|  1.0|    37|
+-----+------+

+-----+-------+
|label|Correct|
+-----+-------+
|  0.0|    154|
|  1.0|     88|
+-----+-------+

Sensitivity: 70.39999999999999
FP Rate: 15.846994535519126
Specificity: 84.15300546448088
Precision: 75.21367521367522
Prevalence: 40.58441558441558


40.58441558441558

In [13]:
println("results from #4 - decision tree of age, sex and pclass")
analysis(predictionBinary4, true)

results from #4 - decision tree of age, sex and pclass
		Predicted
		Y	N
Actual	Y	64.0	55.0
	N	13.0	195.0

Accuracy: 79.20489296636084
Missclassification: 20.795107033639145
+-----+------+
|label|Errors|
+-----+------+
|  0.0|    13|
|  1.0|    55|
+-----+------+

+-----+-------+
|label|Correct|
+-----+-------+
|  0.0|    195|
|  1.0|     64|
+-----+-------+

Sensitivity: 53.78151260504202
FP Rate: 6.25
Specificity: 93.75
Precision: 3200.0
Prevalence: 36.391437308868504


36.391437308868504

<b><em>Analysis: Age alone is a very poor classifier for survivors.  Although a 50% accuracy rate appears high - it is simply due to predicting non-survivors (a broken clock, or guessing exclusively False in a True/False exam).

Adding additional classifiers - sex and class - helps improve our accuracy.  This makes sense, as we add more features, we are better able to classify the data.

There doesn't appear to be a significant difference between Logistic Regression and Decision Tree results.  They have been between 75-80% accuracy.  Unfortunately, both seem to have more False Positives (telling people they will survive when they won't) than False Negatives (telling people they won't survive and then they do).</b></em>